In [8]:
import subprocess

In [16]:
subprocess.Popen(['python3', 'test.py', 'ssd1351'])

In [1]:
import _pickle as pickle

def load_from_file(file_path):
    with open(file_path, "rb") as input_file:
        data = pickle.load(input_file)
    return data

In [5]:
import multiprocessing
from time import sleep

In [4]:
# define worker function
def calculate(process_name, tasks, results):
    print('[%s] evaluation routine starts' % process_name)

    while True:
        new_value = tasks.get()
        if new_value < 0:
            print('[%s] evaluation routine quits' % process_name)

            # Indicate finished
            results.put(-1)
            break
        else:
            # Compute result and mimic a long-running task
            compute = new_value * new_value
            sleep(0.02*new_value)

            # Output which process received the value
            # and the calculation result
            print('[%s] received value: %i' % (process_name, new_value))
            print('[%s] calculated value: %i' % (process_name, compute))

            # Add result to the queue
            results.put(compute)

    return

In [6]:
# Define IPC manager
manager = multiprocessing.Manager()

# Define a list (queue) for tasks and computation results
tasks = manager.Queue()
results = manager.Queue()
# Create process pool with four processes
num_processes = 4
pool = multiprocessing.Pool(processes=num_processes)
processes = []
# Initiate the worker processes
for i in range(num_processes):

    # Set process name
    process_name = 'P%i' % i

    # Create the process, and connect it to the worker function
    new_process = multiprocessing.Process(target=calculate, args=(process_name,tasks,results))

    # Add new process to the list of processes
    processes.append(new_process)

    # Start the process
    new_process.start()
    # Fill task queue
task_list = [43, 1, 780, 256, 142, 68, 183, 334, 325, 3]
for single_task in task_list:
    tasks.put(single_task)

# Wait while the workers process
sleep(5)

[P0] evaluation routine starts
[P1] evaluation routine starts
[P2] evaluation routine starts
[P3] evaluation routine starts
[P1] received value: 1
[P1] calculated value: 1
[P0] received value: 43
[P0] calculated value: 1849
[P0] received value: 68
[P0] calculated value: 4624
[P1] received value: 142
[P1] calculated value: 20164
[P3] received value: 256
[P3] calculated value: 65536
[P0] received value: 183
[P0] calculated value: 33489
[P0] received value: 3
[P0] calculated value: 9
[P1] received value: 334
[P1] calculated value: 111556
[P3] received value: 325
[P3] calculated value: 105625
[P2] received value: 780
[P2] calculated value: 608400
[P0] evaluation routine quits
[P1] evaluation routine quits
[P3] evaluation routine quits
[P2] evaluation routine quits


In [7]:
# Quit the worker processes by sending them -1
for i in range(num_processes):
    tasks.put(-1)

# Read calculation results
num_finished_processes = 0
while True:
    # Read result
    new_result = results.get()

    # Have a look at the results
    if new_result == -1:
        # Process has finished
        num_finished_processes += 1

        if num_finished_processes == num_processes:
            break
    else:
        # Output result
        print('Result:' + str(new_result))

Result:1
Result:1849
Result:4624
Result:20164
Result:65536
Result:33489
Result:9
Result:111556
Result:105625
Result:608400
